<a href="https://colab.research.google.com/github/HudsonHuang/mixbatch/blob/master/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
# from  matplotlib import pyplot as plt
np.random.seed(6) #随机数，这样做的目的是在每次运行程序时，初始值保持一致。seed的值可以随便设置。
import keras
from keras.models import Sequential
from keras.layers import Dense , Dropout ,Activation, Flatten
from keras.layers import Conv2D ,MaxPooling2D   #卷积层，池化层
from keras.utils import np_utils
from keras.datasets import mnist
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
import numpy as np
import keras.backend as K
from keras.layers import Layer
np.random.seed(2019)
tf.set_random_seed(2019)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # 不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)



In [0]:

class MixBatch(Layer):
    def __init__(self,batch_size, alpha=0.2, **kwargs) -> None:
        self.alpha = alpha
        self.batch_size = batch_size
        super().__init__(**kwargs)

    def get_beta(self,input_shape):
        beta_shape = [1] * len(list(input_shape))
        beta_shape[0] = self.batch_size
        return tf.distributions.Beta(self.alpha, self.alpha).sample(beta_shape)

    def mixup_tf(self, features):
        # do mixup here
        # tensorflow version
        print("features",features.shape)
        input_shape = K.int_shape(features)
        mix = self.get_beta(features.shape)
        mix = tf.maximum(mix, 1 - mix)  # contrl to let data close to x1
        features = features * mix + features[::-1] * (1 - mix)
        return features

    def call(self, x,training=None, **kwargs):
        if training:
            return self.mixup_tf(x)
        else:
            return x

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = {
            'alpha': self.alpha,
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [0]:
def get_default_callbacks(log_dir=None,comment=''):
    import time
    import os

    if log_dir==None:
        def _get_current_date():
            strDate = time.strftime('%Y%m%d_%H%M%S',
                                    time.localtime(time.time())) 
            return strDate
        log_dir = os.path.join('./',_get_current_date()+comment)

    cb = []
    weight_path = os.path.join(log_dir, 'model.h5')
    ckpt_cb = keras.callbacks.ModelCheckpoint(weight_path,
                                            save_weights_only=True,
                                            save_best_only=True)
    cb.append(ckpt_cb)

    tb_cb = keras.callbacks.TensorBoard(log_dir=log_dir)
    cb.append(tb_cb)

    # nan_cb = keras.callbacks.TerminateOnNaN()
    # cb.append(nan_cb)

    # update_pruning = sparsity.UpdatePruningStep(),
    # pruning_summary = ssparsity.PruningSummaries(log_dir=log_dir, profile_batch=0)
    # cb.append(update_pruning)
    # cb.append(pruning_summary)
    return cb


In [5]:

(X_train,y_train),(X_test,y_test)=mnist.load_data()   #导入mnist数据
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
#这里大家可能不是很了解，主要是为了把mnist数据变成二维图片形式
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255     #做标准化（0，1）之间 ，深度学习对样本值敏感需要做标准化处理
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
#把标签值变成一维数组显示，例如 标签1 ——0100000000

11493376/11490434 [==============================] - 1s 0us/step


In [0]:

model = Sequential()
model.add(MixBatch(batch_size=32))
model.add(Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=(28,28,1)))
model.add(MixBatch(batch_size=32))
#第一层卷积层，32个特征图，每个特征图中的神经元与输入中的3×3的领域相连。参数可调
model.add(Conv2D(64, (3, 3), activation='relu'))   #第二层卷积层,64个特征图。参数可调
model.add(MixBatch(batch_size=32))
model.add(MaxPooling2D(pool_size=(2, 2)))   #池化层 2×2做映射，参数可调
model.add(MixBatch(batch_size=32))
model.add(Dropout(0.25))   #Dropou算法用于测试集的精确度的优化，这个也可以换。
model.add(Flatten())    #把二维数组拉成一维
model.add(Dense(128, activation='relu'))    #正常的神经网络层
model.add(MixBatch(batch_size=32))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax')) #输出层
model.compile(loss='categorical_crossentropy',
                         optimizer='adam',
                          metrics=['accuracy'])   #交叉熵，训练优化算法adam
cb = get_default_callbacks(comment="MBall")
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, verbose=1,callbacks=cb,validation_data=(X_test, Y_test))  # batch=32,epoch=10,参数可调，总迭代次数大家会算吗？
# score = model.evaluate(, verbose=0)

W0716 02:12:45.124864 139682443163520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0716 02:12:45.129883 139682443163520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
W0716 02:12:47.662666 139682443163520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0716 02:12:47.671231 139682443163520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138:

Train on 60000 samples, validate on 10000 samples


W0716 02:12:50.373660 139682443163520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0716 02:12:50.375442 139682443163520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/10
59232/60000 [============================>.] - ETA: 0s - loss: 0.1990 - acc: 0.9390